### <img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393/578 Máquinas de Aprendizaje - 2019-1 </h1>

<H3 align='center'> Tarea 3  </H3>
<hr style="height:2px;border:none"/>

**Temas**  
* Manipulaciones en numpy
* Arboles de clasificación y regresión
* Algunas Redes neuronales básicas
* Ensamblados, Adaboost y Random Forest
* Nociones de clases desbalanceadas


**Formalidades**  
* Equipos de trabajo de 2 personas (*Ambos estudiantes deben estar preparados para presentar la tarea el día de la entrega*)
* El entregable debe ser un _Jupyter Notebook_ incluyendo los códigos utilizados, los resultados, los gráficos realizados y comentarios. Debe seguir una estructura similar a un informe (se debe introducir los problemas a trabajar, presentar los resultados y discutirlos). Si lo prefiere puede entregar un _Jupyter Notebook_ por pregunta o uno por toda la tarea, con tal de que todos los entregables esten bien identificados y se encuentren en el mismo repositorio de _Github_.
* Se debe preparar una presentación del trabajo realizado y sus hallazgos. El presentador será elegido aleatoriamente y deberá apoyarse en el _Jupyter Notebook_ que entregarán. 
* Formato de entrega: envı́o de link del repositorio *privado* en _Github_, al correo electrónico del ayudante (*<alvaro.valderrama.13@sansano.usm.cl>*), en copia al profesor (*<cvalle@inf.utfsm.cl>*). Especificar el siguiente asunto: [INF393/578-2019 Tarea 3]. Invitar como colaborador al usuario de github "avalderr" para poder acceder al repositorio.
* Fecha de entrega y presentaciones: 13 de Septiembre. Hora límite de entrega: 23:50. Cualquier _commit_ luego de la hora límite no será evaluado. Se realizará descuento por atrasos en envío del mail. 

<hr style="height:2px;border:none"/>

La tarea se divide en tres partes:

[1.](#primero) Regresión para ubicación espacial  
[2.](#segundo) Clasificación con clases desbalanceadas    
[3.](#tercero) Clasificación en texto   

La tarea tiene ejemplos de códigos con los cuales pueden guiarse en gran parte, sin embargo solo son guias y pueden ser creativos al momento de resolver la tarea. Soluciones creativas o elegantes serán valoradas. También en algunas ocaciones se hacen elecciones arbitrarias, ustedes pueden realizar otras elecciones con tal de que haya una pequeña justificación de por qué su elección es mejor o equivalente.
Recuerden intercalar su código con comentarios y con celdas _Markdown_ con los comentarios de la pregunta y con cualquier analisis, fórmula (en $ \LaTeX $) o explicación que les parezca relevante para justificar sus procedimientos. 
Noten que en general cuando se les pide elegir algo o proponer algo no se evaluará mucho la elección en si, en cambio la argumentación detrás de la elección será lo más ponderado.
Si algun modelo se demora demasiado en correr en su maquina, no olvide que puede correr _Jupyter Notebooks_ en _Collab_ de Google, esto puede ser relevante para las maquinas más lentas al momento de realizar exploraciones con _K-folds_ por ejemplo.

<a id="primero"></a>
## 1. Regresión para ubicación espacial

Un problema que tiene aplicaciones para muchos servicios es el calculo de la ubicación espacial exacta de un dispositivo conectado dentro de un edificio o un complejo de edificios. Estando dentro de un aeropuerto nuestro celular nos podría dar indicaciones para llegar a nuestra puerta de embarque o dentro de un mall encontrar la tienda que encontramos serían aplicaciones que fácilmente se desprenden de la solución a este problema. Actualmente en exteriores el problema se soluciona fácilmente mediante el uso de GPS, sin embargo en interiores el problema se vuelve más complicado pues hay considerable pérdida de la señar GPS y muchas superficies de construcción pueden resultar reflectantes para algunos tipos de señales por lo cual el problema es mucho más dificil de modelar que el problema en el exterior. 


En esta primera parte de la tarea utilizaremos un _dataset_ creado en 2013 con más de 20000 mediciones de posición dentro de 4 edificios de la universidad de Jaume, disponible en https://archive.ics.uci.edu/ml/datasets/UJIIndoorLoc junto con una detallada descripción. Nuestra tarea será predecir la ubicación espacial en latitud, longitud, y altura (en pisos) de los distintos usuarios que realizaron esta tarea, a partir de las intensidades de señal medidas por los 520 distintos puntos de acceso inalambricos. Para esto, probaremos utilizaremos arboles de regresión, regresión lineal y redes neuronales, comparando los distintos desempeños y las ventajas y desventajas de cada aproximación. 


#### 1.a Carga de datos

Cargue los datos de entrenamiento y validación y júntelos en un solo DataFrame como muestra el código. 

De los datos entregados luego del _target_ solo consideraremos el tipo de celular utilizado, convirtiendolo a variables _dummies_ como muestra el código. 

Separe el _target_ del resto de los datos.

In [86]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

import scipy.stats as stats

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.utils import shuffle

In [1]:
#from google.colab import files
#uploaded = files.upload()

Saving trainingData.csv to trainingData.csv
Saving validationData.csv to validationData.csv


In [11]:
df_tr = pd.read_csv("data/data_pregunta1/trainingData.csv")
df_val = pd.read_csv("data/data_pregunta1/validationData.csv")

In [12]:
df_tr.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7541.2643,4.864921e+06,2,1,106,2,2,23,1371713733
1,100,100,100,100,100,100,100,100,100,100,...,100,-7536.6212,4.864934e+06,2,1,106,2,2,23,1371713691
2,100,100,100,100,100,100,100,-97,100,100,...,100,-7519.1524,4.864950e+06,2,1,103,2,2,23,1371714095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7524.5704,4.864934e+06,2,1,102,2,2,23,1371713807
4,100,100,100,100,100,100,100,100,100,100,...,100,-7632.1436,4.864982e+06,0,0,122,2,11,13,1369909710


In [13]:
print(df_tr.columns)

Index(['WAP001', 'WAP002', 'WAP003', 'WAP004', 'WAP005', 'WAP006', 'WAP007',
       'WAP008', 'WAP009', 'WAP010',
       ...
       'WAP520', 'LONGITUDE', 'LATITUDE', 'FLOOR', 'BUILDINGID', 'SPACEID',
       'RELATIVEPOSITION', 'USERID', 'PHONEID', 'TIMESTAMP'],
      dtype='object', length=529)


In [55]:
df = shuffle(df_tr.append(df_val))

target_labels = [520,521,253]
# Indices del 1 al 259 sin los indices del target
attribute_labels = [l for l in range(len(df.columns)) if l not in target_labels]

target_cols =  df.columns[target_labels] 
print(target_cols)

Index(['LONGITUDE', 'LATITUDE', 'WAP254'], dtype='object')


In [60]:
# Para eliminar PHONEID ya que se pasará a categorical
phone_index = list(df.columns).index('PHONEID')
attribute_labels.remove(phone_index)

In [61]:
attribute_cols = df.columns[attribute_labels]

x_un = pd.get_dummies(df.PHONEID)
categorical_columns = x_un.columns
x_un[attribute_cols] = df[attribute_cols]

y_u = df[target_cols]

Separe los datos a partir de su índice, dejando un 80% de datos en el _train set_ y un 20% de validación. 
¿Por qué no es mala idea en este caso separar los datos a partir de su índice? ¿Qué problema podría tenerse en caso de no haber hecho `shuffle` al momento de cargar los datos? 


In [63]:
porcentaje_sep = 0.8
indice_sep = math.floor(x_un.shape[0]*0.8)

x_tr_un = x_un.iloc[0:indice_sep]
x_val_un = x_un.iloc[indice_sep:]

y_tr_u = y_u.iloc[0:indice_sep]
y_val_u = y_u.iloc[indice_sep:]

**Comentario**: En caso de no haber hecho shuffle podríamos estar sesgando los sets de training y de validación, en caso de que los datos vinieran ordenados de cierta manera en especifico. Ejemplo: que vinieran ordenados por ubucación, o por fuente de la que se obtuvieron los datos.

#### 1.b Estandarización de los datos

Utilice _Standar Scaler_ para los atributos no categóricos y para los datos categóricos simplemente sustraiga 0.5, como muestra el código. ¿Tiene realmente sentido estandarizar los datos categóricos (binarios) o le parece mejor la aproximación elegida?

Estandarice igualmente el _target_. Cabe mencionar que la tercera componente del target es el piso en el que se encuentra, variable categórica, preferiremos utilizarla como variable continua para ver que tan bien nuestros algoritmos pueden aproximar la ubicación y para facilitar la comparación de distintos modelos. 

In [67]:
scaler_x = StandardScaler()
scaler_x.fit(x_tr_un[attribute_cols])

x_tr = pd.np.concatenate((scaler_x.transform(x_tr_un[attribute_cols]), x_tr_un[categorical_columns].values-0.5), axis=1)
x_val = pd.np.concatenate((scaler_x.transform(x_val_un[attribute_cols]), x_val_un[categorical_columns].values-0.5), axis=1)

scaler_y = StandardScaler()
scaler_y.fit(y_tr_u)

y_tr = scaler_y.transform(y_tr_u)
y_val = scaler_y.transform(y_val_u)

#### 1.c Primer Arbol
Entrenaremos ahora un árbol de regresión con produndidad 3. Como métrica utilizaremos el Error Absoluto Medio o MAE a lo largo del resto de esta pregunta. ¿Qué tan bueno le parece el _score_ (coeficiente $R^2$) obtenido por el regresor?

Compare el desempeño del árbol, tanto en MAE como en _score_, en validación y entrenamiento, con respecto a un regresor lineal tradicional. ¿Cómo se comparan los tiempos de entrenamiento? ¿Le parece justificado el costo adicional?

In [77]:
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(criterion='mae', max_depth=3)
tree_model.fit(x_tr, y_tr)

DecisionTreeRegressor(criterion='mae', max_depth=3, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [80]:
mse_tr = mean_squared_error(tree_model.predict(x_tr),y_tr)
mse_val = mean_squared_error(tree_model.predict(x_val),y_val)

print("MSE TRAINING: ", mse_tr)
print("MSE VALIDATION: ", mse_val)

MSE TRAINING:  0.05627276637970452
MSE VALIDATION:  0.05212937244745997


In [85]:
# Variance weighted
score_tr = tree_model.score(x_tr,y_tr)
score_val = tree_model.score(x_val,y_val)

print("SCORE TRAINING: ", score_tr)
print("SCORE VALIDATION: ", score_val)

SCORE TRAINING:  0.9437272336202955
SCORE VALIDATION:  0.9556772157227131


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:420: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

In [121]:
# Ahora si ordenamos todos los resultados y usamos todas las medidas de score que existe, para comparar

tree_results = list()
tree_results.append(["MEDIDA", "TREE DEPTH 3 MODEL"])

# MSE
mse_tr = mean_squared_error(tree_model.predict(x_tr),y_tr)
mse_val = mean_squared_error(tree_model.predict(x_val),y_val)
tree_results.append(["MSE TRAINING",  round(mse_tr,4)])
tree_results.append(["MSE VALIDATION",  round(mse_val,4)])

# Uniform average
score_tr = r2_score(tree_model.predict(x_tr), y_tr)
score_val = r2_score(tree_model.predict(x_val), y_val)
tree_results.append(["R2_SCORE TRAINING (Uniform average)",  round(score_tr,4)])
tree_results.append(["R2_SCORE VALIDATION (Uniform average)",  round(score_val,4)])

# Variance weighted
score_tr = r2_score(tree_model.predict(x_tr), y_tr, multioutput="variance_weighted")
score_val = r2_score(tree_model.predict(x_val), y_val, multioutput="variance_weighted")
tree_results.append(["SCORE TRAINING (Variance weighted)",  round(score_tr,4)])
tree_results.append(["SCORE VALIDATION (Variance weighted)",  round(score_val,4)])

# Raw values
score_tr = r2_score(tree_model.predict(x_tr), y_tr, multioutput="raw_values")
score_val = r2_score(tree_model.predict(x_val), y_val, multioutput="raw_values")
tree_results.append(["SCORE TRAINING (Raw values)",  list(map(lambda x: round(x,4), score_tr))])
tree_results.append(["SCORE VALIDATION (Raw values)",  list(map(lambda x: round(x,4), score_val))])

In [122]:
# Display tabular de los resultados

from IPython.display import HTML, display
import tabulate

print(tabulate.tabulate(tree_results, headers="firstrow"))
#display(HTML(tabulate.tabulate(stump_results, tablefmt='html')))

MEDIDA                                 TREE DEPTH 3 MODEL
-------------------------------------  ------------------------
MSE TRAINING                           0.0563
MSE VALIDATION                         0.0521
R2_SCORE TRAINING (Uniform average)    0.9374
R2_SCORE VALIDATION (Uniform average)  0.9423
SCORE TRAINING (Variance weighted)     0.9409
SCORE VALIDATION (Variance weighted)   0.9538
SCORE TRAINING (Raw values)            [0.9577, 0.8727, 0.9818]
SCORE VALIDATION (Raw values)          [0.9596, 0.8716, 0.9957]


#### 1.d _Stump_
Entrene ahora un _Stump_ o arbol de profundidad 1. ¿Cómo cambia el error (MAE) en entrenamiento y validación con respecto al árbol de la pregunta anterior? Discuta sobre el numero de parámetros entrenables que tendrá cada modelo.

In [93]:
stump_model =  DecisionTreeRegressor(criterion='mae', max_depth=1)
stump_model.fit(x_tr, y_tr)

DecisionTreeRegressor(criterion='mae', max_depth=1, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [119]:
# Ordenamos resultados

stump_results = list()
stump_results.append(["MEDIDA", "STUMP MODEL"])

# MSE
mse_tr = mean_squared_error(stump_model.predict(x_tr),y_tr)
mse_val = mean_squared_error(stump_model.predict(x_val),y_val)
stump_results.append(["MSE TRAINING",  round(mse_tr,4)])
stump_results.append(["MSE VALIDATION",  round(mse_val,4)])

# Uniform average
score_tr = r2_score(stump_model.predict(x_tr), y_tr)
score_val = r2_score(stump_model.predict(x_val), y_val)
stump_results.append(["R2_SCORE TRAINING (Uniform average)",  round(score_tr,4)])
stump_results.append(["R2_SCORE VALIDATION (Uniform average)",  round(score_val,4)])

# Variance weighted
score_tr = r2_score(stump_model.predict(x_tr), y_tr, multioutput="variance_weighted")
score_val = r2_score(stump_model.predict(x_val), y_val, multioutput="variance_weighted")
stump_results.append(["SCORE TRAINING (Variance weighted)",  round(score_tr,4)])
stump_results.append(["SCORE VALIDATION (Variance weighted)",  round(score_val,4)])

# Raw values
score_tr = r2_score(stump_model.predict(x_tr), y_tr, multioutput="raw_values")
score_val = r2_score(stump_model.predict(x_val), y_val, multioutput="raw_values")
stump_results.append(["SCORE TRAINING (Raw values)",  list(map(lambda x: round(x,4), score_tr))])
stump_results.append(["SCORE VALIDATION (Raw values)",  list(map(lambda x: round(x,4), score_val))])

In [120]:
# Display tabular de los resultados

from IPython.display import HTML, display
import tabulate

print(tabulate.tabulate(stump_results,  headers="firstrow"))
#display(HTML(tabulate.tabulate(stump_results, tablefmt='html')))

MEDIDA                                 STUMP MODEL
-------------------------------------  -----------------------------------------
MSE TRAINING                           0.5668
MSE VALIDATION                         0.7462
R2_SCORE TRAINING (Uniform average)    -1.9825959122625624e+27
R2_SCORE VALIDATION (Uniform average)  -3.457307632128715e+28
SCORE TRAINING (Variance weighted)     -0.078
SCORE VALIDATION (Variance weighted)   -0.4117
SCORE TRAINING (Raw values)            [0.6525, 0.4446, -5.947787736787687e+27]
SCORE VALIDATION (Raw values)          [0.6567, 0.4398, -1.0371922896386145e+29]


#### 1.e Profundidad máxima
Explore distintos valores de la profundidad máxima del árbol. Para esto, realice K-fold sobre al menos 10 valores distintos de la profundidad del árbol y compare el promedio de los MAE de validación. También guarde los promedios de MAE de _train_ y grafique como estos varían a medida varia la profundidad máxima. Idealmente debería considerar un rango entre profundidad máxima 1 y 20. Note que este paso puede resultar largo en tiempo de cómputo, podría ser recomendable correrlo en _Collab_ en caso de tener máquinas más lentas. 

Finalmente, entrene el arbol con profundidad óptima sobre todos los datos de entrenamiento y calcule su MAE sobre validación. ¿Mejoramos el desempeño obtenido anteriormente con los árboles sin sintonizar? 

In [ ]:
from sklearn.model_selection import KFold

X_tr_kfold = x_tr.append(x_val)
y_tr_kfold = y_tr.append(y_val)
kf = KFold(n_splits=5)
print(kf)

In [ ]:
depth_values = list(range(2,12))
scores_by_depth_tr = list()
scores_by_depth_val = list()
errors_by_depth_tr = list()
errors_by_depth_tr = list()

for depth_i in depth_values:
    fold_scores_tr = list()
    fold_scores_val = list()
    fold_errors_tr = list()
    fold_errors_val = list()
    for train_index, test_index in kf.split(X_tr_kfold):       
        X_train_cross, X_test_cross = X_tr_kfold.iloc[train_index,:], X_tr_kfold.iloc[test_index,:]
        y_train_cross, y_test_cross = y_tr_kfold.iloc[train_index,:], y_tr_kfold.iloc[test_index,:]

        # Entreno del modelo
        kfold_model = DecisionTreeRegressor(criterion='mae', max_depth=depth_i)
        kfold_model.fit(X_train_cross, y_train_cross)
        
        # Training Error
        error_tr = mean_squared_error(kfold_model.predict(X_train_cross), y_train_cross)
        # Validation Error
        error_val = mean_squared_error(kfold_model.predict(X_test_cross), y_test_cross)
        # Training Score 
        score_tr = r2_score(tree_model.predict(x_tr), y_tr)
        # Validation Score
        score_val = r2_score(tree_model.predict(x_val), y_val)
        
        # Guardar resultados de este fold
        fold_errors_tr.append(error_tr)
        fold_errors_val.append(error_val)
        fold_scores_tr.append(score_tr)
        fold_scores_val.append(score_val)
        
    errors_by_depth_tr.append((sum(fold_errors_tr)/len(ffold_errors_tr)))
    errors_by_depth_tr.append((sum(fold_errors_val)/len(fold_errors_val)))
    scores_by_depth_tr.append((sum(fold_scores_tr)/len(fold_scores_tr)))
    scores_by_depth_val.append((sum(fold_scores_val)/len(fold_scores_val)))

#### 1.f Random Forest

En la pregunta anterior vimos que una profundidad de árbol muy grande puede llevarnos a _overffiting_, pues la cantidad de "decisiones" que puede aprender el modelo aumentan significativamente y el modelo tenderá a aprender de memoria el conjunto de entrenamiento, perdiendo capacidad de generalización. Otra aproximación para evitar _overffiting_ no utilizar todos los datos para entrenar, estrategia implementada por _Random Forest_. 

Discuta por qué el utilizar muchos modelos que no conocen todos los datos puede ayudar a mejorar la capacidad de generalización, evitando así que el modelo final este sobreajustado. Entrene luego un _Random Forest Regressor_, con a lo menos 10 árboles. Discuta sobre que elección realizar sobre la profundidad máxima de estos árboles. ¿Cómo se compara el desempeño de _Random Forest_ frente a un árbol por si solo?

In [0]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=20,
                               criterion='mae',
                               max_depth=5)

#### 1.g Otros parámetros

Por último, investigue un poco sobre los otros parámetros de los árboles y sus implicancias. Luego, entrene un arbol modificando algunos de sus parámetros de una forma que piensen podría haber mejorado el desempeño. Discuta

In [0]:
tree = DecisionTreeRegressor(criterion='mae',
                            max_depth=None,
                            min_impurity_decrease=0.1, # for instance
                            min_samples_split=int(x_tr.shape[0]*0.05) #for instance
                            )